In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
import re

In [24]:
text = """
The quick brown fox jumped over the lazy dog.
The fox is quick and brown. The dog is lazy.
The cat is neither quick nor lazy.
"""

def preprocess(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

tokens = preprocess(text)
vocab = list(set(tokens))

word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for word, i in word_to_ix.items()}

In [27]:
def get_context_pairs(tokens, window_size=2):
    pairs = []
    for i, target in enumerate(tokens):
        start = max(0, i - window_size) # to not look before the start
        end = min(len(tokens), i + window_size + 1) # to not look after the end
        context = tokens[start:i] + tokens[i+1:end] # context before + after of the "target"
        pairs.extend([(target, ctx) for ctx in context])
    return pairs

context_pairs = get_context_pairs(tokens)